In [ ]:
# Standard IPython notebook imports
%matplotlib inline

import os
from io import StringIO

import graphviz
import httpimport
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as sstats
import seaborn as sns
# import sklearn.ensemble
# import sklearn.tree

from tqdm import tqdm_notebook

# In this way we can import functions straight from github
with httpimport.github_repo(
    "janchorowski", "nn_assignments", module="common", branch="nn18"
):
    from common.plotting import plot_mat

sns.set_style("whitegrid")
from sklearn import feature_selection
import json

import copy

In [ ]:
leagues = ["DIAMOND"]#["BRONZE","GOLD","PLATINUM","SILVER","DIAMOND"]
paths = [f"/content/eun1_{d}_RANKED_SOLO_5x5.csv" for d in leagues]
# for p in paths:
#     print(p)
#     with open (p, "r") as myfile:
#         data=myfile.readlines()
#         print(len(data[0]))

dfs = [pd.read_csv(path) for path in paths]

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (44,45,46,47,53,54,55,56,62,63,64,65) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (53,54,55,56) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (44,45,46,47) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [ ]:
with open('/content/champs_snd_version.json') as f:
  champion_data_raw = json.load(f)['data']
# print(champion_data_raw)

champion_data = {} # key -> {'name' :foo, 'attributes' :[bar] }
all_attributes = set()
attr_sets = set()
for _, data in champion_data_raw.items():
    name = data['name']
    key = int(data['key'])
    tags = data['tags']
    attr_sets.add(tuple(tags))
    champion_data[key] = {'name' : name, 'attributes': tags}
    for a in tags:
        all_attributes.add(a)
    # print(name,key,tags)
print(all_attributes)
attr_enc = dict(zip(all_attributes,range(6)))
print(attr_enc)
print(attr_sets)

{'Support', 'Fighter', 'Assassin', 'Marksman', 'Tank', 'Mage'}
{'Support': 0, 'Fighter': 1, 'Assassin': 2, 'Marksman': 3, 'Tank': 4, 'Mage': 5}
{('Mage', 'Support'), ('Support', 'Mage'), ('Mage', 'Assassin'), ('Mage', 'Fighter'), ('Fighter', 'Mage'), ('Assassin', 'Mage'), ('Tank', 'Mage'), ('Mage', 'Marksman'), ('Fighter', 'Tank'), ('Tank',), ('Fighter', 'Support'), ('Support', 'Assassin'), ('Support', 'Fighter'), ('Assassin',), ('Fighter',), ('Marksman',), ('Marksman', 'Assassin'), ('Fighter', 'Marksman'), ('Mage',), ('Marksman', 'Mage'), ('Tank', 'Fighter'), ('Tank', 'Support'), ('Support',), ('Marksman', 'Support'), ('Fighter', 'Assassin'), ('Assassin', 'Fighter'), ('Support', 'Tank')}


In [ ]:
def df_with_champs(df):   
    df_champs = copy.deepcopy(df)
    id_cols = list(filter(lambda x: x.endswith("championId"),df_champs.columns))
    for column in id_cols:
        col_name = column[:-2] + "_name"
        col_attr = column[:-2] + "_attribute"
        df_champs[col_name] = df[column].apply(lambda x: champion_data[x]['name'])
        df_champs[col_attr] = df[column].apply(lambda x: champion_data[x]['attributes'][0]) #comment if u dont want attributes
    df_champs = df_champs.drop(columns=id_cols)
    return df_champs

In [ ]:
def tidy_df_up(df):
    renaming_dic = dict({"1_result" : "1_win"}, **dict(zip(df.columns[3:],list(map(lambda x : x[0] + "_" + x[3:],df.columns[3:])))))
    df = df.rename(columns=renaming_dic)
    df.iloc[:,1] = df.iloc[:,1] == "Win"
    df = df.dropna()
    ids = list(filter(lambda s: s.endswith("Id"),df.columns))
    df = df.astype(dict(zip(ids,np.repeat("int32",len(ids)))))
    return df

In [ ]:
for df in dfs:
    print(len(df),len(tidy_df_up(df)))

16800 9500
18820 11129
22572 12734
16638 9760
30700 16279


In [ ]:
for df,le in zip(dfs,leagues):
    df2 = tidy_df_up(df)
    df2.to_csv(f"/content/eun1_{le}_RANKED_SOLO_CLEAN.csv",index=False)

In [ ]:
df = pd.read_csv("/content/eun1_DIAMOND_RANKED_SOLO_BEST.csv")
# df2= tidy_df_up(df)
df2.head()

,match_id,1_win,1_firstDragon,1_MIDDLE_xp_per_min,1_MIDDLE_creeps_per_min,1_MIDDLE_gold_per_min,1_MIDDLE_damage_taken_per_min,1_MIDDLE_first_blood,1_MIDDLE_first_blood_assist,1_MIDDLE_first_tower,1_MIDDLE_first_tower_assist,1_TOP_xp_per_min,1_TOP_creeps_per_min,1_TOP_gold_per_min,1_TOP_damage_taken_per_min,1_TOP_first_blood,1_TOP_first_blood_assist,1_TOP_first_tower,1_TOP_first_tower_assist,1_SUPPORT_xp_per_min,1_SUPPORT_creeps_per_min,1_SUPPORT_gold_per_min,1_SUPPORT_damage_taken_per_min,1_SUPPORT_first_blood,1_SUPPORT_first_blood_assist,1_SUPPORT_first_tower,1_SUPPORT_first_tower_assist,1_JUNGLE_xp_per_min,1_JUNGLE_creeps_per_min,1_JUNGLE_gold_per_min,1_JUNGLE_damage_taken_per_min,1_JUNGLE_first_blood,1_JUNGLE_first_blood_assist,1_JUNGLE_first_tower,1_JUNGLE_first_tower_assist,1_ADC_xp_per_min,1_ADC_creeps_per_min,1_ADC_gold_per_min,1_ADC_damage_taken_per_min,1_ADC_first_blood,...,2_JUNGLE_first_blood,2_JUNGLE_first_blood_assist,2_JUNGLE_first_tower,2_JUNGLE_first_tower_assist,2_SUPPORT_xp_per_min,2_SUPPORT_creeps_per_min,2_SUPPORT_gold_per_min,2_SUPPORT_damage_taken_per_min,2_SUPPORT_first_blood,2_SUPPORT_first_blood_assist,2_SUPPORT_first_tower,2_SUPPORT_first_tower_assist,2_ADC_xp_per_min,2_ADC_creeps_per_min,2_ADC_gold_per_min,2_ADC_damage_taken_per_min,2_ADC_first_blood,2_ADC_first_blood_assist,2_ADC_first_tower,2_ADC_first_tower_assist,1_MIDDLE_champion_name,1_MIDDLE_champion_attribute,1_TOP_champion_name,1_TOP_champion_attribute,1_SUPPORT_champion_name,1_SUPPORT_champion_attribute,1_JUNGLE_champion_name,1_JUNGLE_champion_attribute,1_ADC_champion_name,1_ADC_champion_attribute,2_MIDDLE_champion_name,2_MIDDLE_champion_attribute,2_TOP_champion_name,2_TOP_champion_attribute,2_JUNGLE_champion_name,2_JUNGLE_champion_attribute,2_SUPPORT_champion_name,2_SUPPORT_champion_attribute,2_ADC_champion_name,2_ADC_champion_attribute
0,2725511175,True,False,410.5,7.4,293.4,430.0,True,False,False,False,495.9,6.7,261.3,510.7,False,False,False,False,271.8,0.0,139.8,238.3,False,False,False,False,355.2,0.2,331.4,822.5,False,False,False,False,281.9,6.8,238.3,219.0,False,...,False,False,False,False,278.6,0.2,195.6,199.5,False,False,False,False,340.8,9.4,356.3,186.4,False,False,False,True,Lucian,Marksman,Garen,Fighter,Lulu,Support,Nidalee,Assassin,Vayne,Marksman,Kalista,Marksman,Viego,Assassin,Graves,Marksman,Janna,Support,Aphelios,Marksman
1,2710306824,False,True,465.0,7.1,328.0,450.8,False,False,False,False,375.2,4.0,403.4,546.6,False,False,True,False,239.6,1.2,228.0,296.1,False,False,False,False,355.7,1.0,312.5,630.6,False,False,False,False,274.2,5.5,298.5,614.0,False,...,False,False,False,False,320.7,1.3,423.3,470.5,True,False,False,False,304.6,5.1,370.4,599.6,False,False,False,False,Anivia,Mage,Kled,Fighter,Thresh,Support,Graves,Marksman,Jhin,Marksman,Tristana,Marksman,Fiora,Fighter,Fiddlesticks,Mage,Senna,Marksman,Tahm Kench,Support
2,2677276686,False,True,437.9,5.5,354.6,578.9,False,False,False,False,387.5,5.0,210.4,337.7,False,False,False,False,265.1,2.0,405.9,340.4,True,False,False,False,295.7,0.7,316.6,1022.4,False,False,False,False,306.5,6.4,262.7,415.7,False,...,False,False,False,False,289.5,1.4,233.2,306.8,False,False,False,False,292.0,6.1,382.2,330.7,False,False,False,False,Pantheon,Fighter,Malphite,Tank,Pyke,Support,Nunu & Willump,Tank,Ezreal,Marksman,Orianna,Mage,Darius,Fighter,Olaf,Fighter,Soraka,Support,Kalista,Marksman
3,2713845795,True,True,403.3,5.7,270.3,426.6,False,False,False,False,500.1,5.8,282.2,262.5,False,False,False,False,266.9,0.1,236.5,170.5,False,False,False,False,378.9,0.7,341.9,1043.1,False,False,False,False,244.7,6.1,342.5,576.6,True,...,False,False,False,False,281.6,0.5,220.4,377.3,False,False,False,False,404.0,0.6,388.0,886.7,False,False,False,False,Talon,Assassin,Urgot,Fighter,Yuumi,Support,Fiddlesticks,Mage,Vayne,Marksman,Viktor,Mage,Renekton,Fighter,Yone,Assassin,Vel'Koz,Mage,Rengar,Assassin
4,2596012070,False,False,444.0,7.2,276.0,294.8,False,False,False,False,459.8,6.6,311.5,459.7,False,False,False,Fals

In [ ]:
with open ("/content/eun1_PLATINUM_RANKED_SOLO_CLEAN.csv", "r") as myfile:
        data=myfile.readlines()
        print(data[0])

match_id,1_win,1_firstDragon,1_SUPPORT_championId,1_SUPPORT_xp_per_min,1_SUPPORT_creeps_per_min,1_SUPPORT_gold_per_min,1_SUPPORT_damage_taken_per_min,1_SUPPORT_first_blood,1_SUPPORT_first_blood_assist,1_SUPPORT_first_tower,1_SUPPORT_first_tower_assist,1_ADC_championId,1_ADC_xp_per_min,1_ADC_creeps_per_min,1_ADC_gold_per_min,1_ADC_damage_taken_per_min,1_ADC_first_blood,1_ADC_first_blood_assist,1_ADC_first_tower,1_ADC_first_tower_assist,1_MIDDLE_championId,1_MIDDLE_xp_per_min,1_MIDDLE_creeps_per_min,1_MIDDLE_gold_per_min,1_MIDDLE_damage_taken_per_min,1_MIDDLE_first_blood,1_MIDDLE_first_blood_assist,1_MIDDLE_first_tower,1_MIDDLE_first_tower_assist,1_JUNGLE_championId,1_JUNGLE_xp_per_min,1_JUNGLE_creeps_per_min,1_JUNGLE_gold_per_min,1_JUNGLE_damage_taken_per_min,1_JUNGLE_first_blood,1_JUNGLE_first_blood_assist,1_JUNGLE_first_tower,1_JUNGLE_first_tower_assist,1_TOP_championId,1_TOP_xp_per_min,1_TOP_creeps_per_min,1_TOP_gold_per_min,1_TOP_damage_taken_per_min,1_TOP_first_blood,1_TOP_first_bl

In [ ]:
df = pd.read_csv("/content/eun1_GOLD_RANKED_SOLO_5x5.csv")
print(len(df))
df = tidy_df_up(df)
print(len(df))
df.to_csv(f"kurwix.csv",index=False)
df = pd.read_csv("kurwix.csv")
print(len(df))


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (53,54,55,56) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


18820
11129
11129
